In [2]:
import json
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import importlib as il
import warnings
warnings.filterwarnings('ignore')
# from ipynb.fs.full.data_getter import get_data,splitter

In [3]:
counties = ['LOGAN','LA MOURE','DICKEY','MCINTOSH']
df = pd.read_csv("crop_county_data.csv")

df[df.value == '                 (D)'] = np.NaN
df.dropna()
df['value'] = df['value'].str.replace(',','')
print(df.tail())
df['value'] = df['value'].astype('float64')

df = df[df['county_name'].isin(counties)]
df.loc[df['commodity_desc'] != 'CORN', 'commodity_desc'] = 'NOT CORN'
print(df['commodity_desc'].unique())
df

        Unnamed: 0    year  fips_code state_alpha     county_name  \
103242    103242.0  2020.0    38998.0          ND  OTHER COUNTIES   
103243    103243.0  2020.0    38998.0          ND  OTHER COUNTIES   
103244    103244.0  2020.0    38998.0          ND  OTHER COUNTIES   
103245    103245.0  2020.0    38998.0          ND  OTHER COUNTIES   
103246    103246.0  2020.0    38998.0          ND  OTHER COUNTIES   

       commodity_desc unit_desc   value  \
103242       SOYBEANS     ACRES   44750   
103243      SUNFLOWER     ACRES  144900   
103244          WHEAT     ACRES  171100   
103245          WHEAT     ACRES  251080   
103246          WHEAT     ACRES   33000   

                                           short_desc  
103242                     SOYBEANS - ACRES HARVESTED  
103243          SUNFLOWER, OIL TYPE - ACRES HARVESTED  
103244  WHEAT, SPRING, (EXCL DURUM) - ACRES HARVESTED  
103245         WHEAT, SPRING, DURUM - ACRES HARVESTED  
103246                WHEAT, WINTER - ACRES HA

,Unnamed: 0,year,fips_code,state_alpha,county_name,commodity_desc,unit_desc,value,short_desc
69,69.0,1919.0,38021.0,ND,DICKEY,NOT CORN,ACRES,31000.0,BARLEY - ACRES HARVESTED
70,70.0,1919.0,38021.0,ND,DICKEY,CORN,ACRES,31000.0,"CORN, GRAIN - ACRES HARVESTED"
71,71.0,1919.0,38021.0,ND,DICKEY,NOT CORN,ACRES,12000.0,FLAXSEED - ACRES HARVESTED
72,72.0,1919.0,38021.0,ND,DICKEY,NOT CORN,ACRES,11000.0,"HAY, ALFALFA - ACRES HARVESTED"
73,73.0,1919.0,38021.0,ND,DICKEY,NOT CORN,ACRES,33000.0,OATS - ACRES HARVESTED
...,...,...,...,...,...,...,...,...,...
103065,103065.0,2020.0,38051.0,ND,MCINTOSH,NOT CORN,ACRES,1130.0,OATS - ACRES HARVESTED
103066,103066.0,2020.0,38051.0,ND,MCINTOSH,NOT CORN,ACRES,115700.0,SOYBEANS - ACRES HARVESTED
103067,103067.0,2020.0,38051.0,ND,MCINTOSH,NOT CORN,ACRES,14300.0,"SUNFLOWER, OIL TYPE - ACRES HARVESTED"
103068,103068.0,2020.0,38051.0,ND,MCINTOSH,NOT CORN,ACRES,50700.0,"WHEAT, SPRING, (EXCL DURUM) - ACRES HARVESTED"


In [4]:
df.drop(['fips_code', 'state_alpha', 'unit_desc','short_desc','Unnamed: 0'],axis = 1,inplace = True)

In [5]:
grouped = df.groupby(['year','county_name','commodity_desc']).agg('sum')

In [6]:
grouped.index

MultiIndex([(1919.0,   'DICKEY',     'CORN'),
            (1919.0,   'DICKEY', 'NOT CORN'),
            (1919.0, 'LA MOURE',     'CORN'),
            (1919.0, 'LA MOURE', 'NOT CORN'),
            (1919.0,    'LOGAN',     'CORN'),
            (1919.0,    'LOGAN', 'NOT CORN'),
            (1919.0, 'MCINTOSH',     'CORN'),
            (1919.0, 'MCINTOSH', 'NOT CORN'),
            (1920.0,   'DICKEY',     'CORN'),
            (1920.0,   'DICKEY', 'NOT CORN'),
            ...
            (2019.0,    'LOGAN', 'NOT CORN'),
            (2019.0, 'MCINTOSH', 'NOT CORN'),
            (2020.0,   'DICKEY',     'CORN'),
            (2020.0,   'DICKEY', 'NOT CORN'),
            (2020.0, 'LA MOURE',     'CORN'),
            (2020.0, 'LA MOURE', 'NOT CORN'),
            (2020.0,    'LOGAN',     'CORN'),
            (2020.0,    'LOGAN', 'NOT CORN'),
            (2020.0, 'MCINTOSH',     'CORN'),
            (2020.0, 'MCINTOSH', 'NOT CORN')],
           names=['year', 'county_name', 'commodity_desc'], len

In [7]:
grouped.columns


Index(['value'], dtype='object')

In [8]:
grouped.loc[(1919,)]

value
county_name commodity_desc          
DICKEY      CORN             31000.0
            NOT CORN        301000.0
LA MOURE    CORN             18000.0
            NOT CORN        365000.0
LOGAN       CORN              2000.0
            NOT CORN        208000.0
MCINTOSH    CORN              5000.0
            NOT CORN        237000.0

In [9]:
grouped.to_csv('corn_not_corn.csv')

In [10]:
fg = pd.read_csv('corn_not_corn.csv')

In [110]:
fg.head()

,year,county_name,commodity_desc,value
0,1919.0,DICKEY,CORN,31000.0
1,1919.0,DICKEY,NOT CORN,301000.0
2,1919.0,LA MOURE,CORN,18000.0
3,1919.0,LA MOURE,NOT CORN,365000.0
4,1919.0,LOGAN,CORN,2000.0


In [147]:
corn = fg.loc[fg['commodity_desc']== 'CORN','value']
not_corn = fg.loc[fg['commodity_desc']!= 'CORN','value']

In [150]:
corn.sum()

16743140.0

1      301000.0
3      365000.0
5      208000.0
7      237000.0
9      287000.0
         ...   
793     65840.0
795    140480.0
797    210570.0
799    141180.0
801    183920.0
Name: value, Length: 405, dtype: float64

In [142]:
corn

0    31000.0
2    18000.0
4     2000.0
6     5000.0
8    47000.0
      ...   
3        1.0
4        1.0
5        1.0
6        1.0
7        1.0
Length: 405, dtype: float64

In [28]:
corn = fg[fg['commodity_desc'] == 'CORN']
not_corn = fg[fg['commodity_desc'] != 'CORN']
county = fg.groupby(['year','county_name']).sum()
county.head()

value
year   county_name          
1919.0 DICKEY       332000.0
       LA MOURE     383000.0
       LOGAN        210000.0
       MCINTOSH     242000.0
1920.0 DICKEY       334000.0

In [30]:
joined = fg.join(county,on = ['year','county_name'],how = 'outer',rsuffix = '_')


In [31]:
joined.head()

,year,county_name,commodity_desc,value,value_
0,1919.0,DICKEY,CORN,31000.0,332000.0
1,1919.0,DICKEY,NOT CORN,301000.0,332000.0
2,1919.0,LA MOURE,CORN,18000.0,383000.0
3,1919.0,LA MOURE,NOT CORN,365000.0,383000.0
4,1919.0,LOGAN,CORN,2000.0,210000.0


In [33]:
fg['percent'] = joined['value']/joined['value_']

In [34]:
fg.head()

,year,county_name,commodity_desc,value,percent
0,1919.0,DICKEY,CORN,31000.0,0.093373
1,1919.0,DICKEY,NOT CORN,301000.0,0.906627
2,1919.0,LA MOURE,CORN,18000.0,0.046997
3,1919.0,LA MOURE,NOT CORN,365000.0,0.953003
4,1919.0,LOGAN,CORN,2000.0,0.009524


In [35]:
fg.to_csv('corn_not_corn%.csv')

In [36]:
fg[fg.percent == 1]

,year,county_name,commodity_desc,value,percent
748,2012.0,LOGAN,CORN,63280.0,1.0
755,2013.0,LOGAN,NOT CORN,94000.0,1.0
758,2014.0,LA MOURE,NOT CORN,297500.0,1.0
759,2014.0,LOGAN,NOT CORN,4540.0,1.0
768,2015.0,MCINTOSH,CORN,6910.0,1.0
785,2018.0,DICKEY,NOT CORN,260780.0,1.0
786,2018.0,LA MOURE,NOT CORN,344570.0,1.0
787,2018.0,LOGAN,NOT CORN,53300.0,1.0
788,2018.0,MCINTOSH,NOT CORN,69500.0,1.0
789,2019.0,DICKEY,NOT CORN,149190.0,1.0
